In [5]:
import os
import sqlite3
import shutil

In [6]:
conn = sqlite3.connect('../data/model_results.db')
c = conn.cursor()
c.execute('''
SELECT * FROM results 
WHERE true_label IS NOT NULL 
AND (NOT (true_label = prediction)
OR confidence < 0.75);
''')
rows = c.fetchall()

In [7]:
# c.execute('''CREATE TABLE training (
#     utc_datetime text NOT NULL PRIMARY KEY,
#     datetime text NOT NULL, 
#     file_name text NOT NULL, 
#     predicton text NOT NULL, 
#     confidence real NOT NULL, 
#     true_label text)''')
# conn.commit()

In [8]:
# Number of new rows for training data
len(rows)

3567

In [11]:
# Iterate over the results above
# If the calssification is wrong, copy it 
for i, row in enumerate(rows):
    utc_datetime, datetime, file_name, prediction, confidence, label = row
    num = i+100
    shutil.copy(f'../imgs/{file_name}', f'../imgs/training/{label}_{datetime}.jpg')
    if prediction == 'none' and label =='none' and confidence >= 0.75:
        os.remove(f'../imgs/{file_name}')
        c.execute('''
                  DELETE FROM results 
                  WHERE utc_datetime = ?;
                  ''',
                 (utc_datetime,))

In [12]:
conn.commit()
conn.close()

In [ ]:
os.listdir()